In [1]:
import os 
import sys 
import pandas as pd 
import re
import pickle
import json
import sqlite3
from pprint import pprint
import datetime
import pdb

In [2]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials, SpotifyOAuth

In [3]:
with open('../credentials/spotify_creds.pkl', 'rb') as hnd:
    credentials = pickle.load(hnd)

In [4]:
scope = "user-read-recently-played"

In [6]:
os.environ.update(credentials)

In [7]:
spotify = spotipy.Spotify(client_credentials_manager= SpotifyOAuth(scope=scope
                                                                   , username='malchemist02'))

## Start Thinking about DB 

In [110]:
db_location = 'data/listening_history.db'

In [277]:
con = sqlite3.connect(db_location)
cursor = con.cursor()

In [88]:
cursor.execute("""
SELECT 
    name
FROM 
    sqlite_master
WHERE 
    type ='table' AND 
    name NOT LIKE 'sqlite_%';
""").fetchall()

[('Listening_History',),
 ('Artists_Info',),
 ('Metrics_WoW',),
 ('Songs_by_date_WoW',),
 ('Playlist_IDs',)]

In [89]:
two_weeks_ago = datetime.datetime.now() - datetime.timedelta(days=21)
two_weeks_ago = str(two_weeks_ago.date())
two_weeks_ago

'2021-12-04'

In [116]:
df1 = pd.read_sql(f"""
select * from Listening_History 
where played_at_date > '{two_weeks_ago}'
""", con)
print("Shape: ", df1.shape)
df1.head()


Shape:  (749, 11)


,index,name,artist_name,played_at_date,played_at_time,duration_min,popularity,song_uri,artist_id,playlist_id,after_ts
0,0,Bella Ciao - Música Original de la Serie la Ca...,Manu Pilas,2021-12-05,09:43,2.330733,71,spotify:track:3lWzVNe1yFZlkeBBzUuZYu,spotify:artist:2TJHmhbmT7L3gw2NKyDTHh,None,1638711834460
1,1,Bella Ciao - Música Original de la Serie la Ca...,Manu Pilas,2021-12-05,09:43,2.330733,71,spotify:track:3lWzVNe1yFZlkeBBzUuZYu,spotify:artist:2TJHmhbmT7L3gw2NKyDTHh,None,1638711834460
2,0,Bella Ciao - Música Original de la Serie la Ca...,Manu Pilas,2021-12-05,13:21,2.330733,71,spotify:track:3lWzVNe1yFZlkeBBzUuZYu,spotify:artist:2TJHmhbmT7L3gw2NKyDTHh,None,1638724911098
3,0,Santa Baby,Eartha Kitt,2021-12-05,18:49,3.441767,75,spotify:track:1vZKP9XURuqMp1SpXGnoyb,spotify:artist:1AwO9pWEBSBoWdEZu28XDC,None,1638744583446
4,0,Barrunto,Willie Colón,2021-12-05,20:14,5.602000,41,spotify:track:0oCentzpU5hprEWJtggVIc,spotify:artist:7x5Slu7yTE5icZjNsc3OzW,spotify:playlist:4teNNy1rJvRRa2qyL1Yj0A,1638749658272


In [117]:
df1['playlist_id'].isna().sum()

160

In [189]:
unique_playlists = df1['playlist_id'].dropna().unique()

In [278]:
df2 = pd.read_sql(f"""
select playlist_id, count(*) as num_songs_listened
, count(distinct  song_uri) as unique_songs_listened
, sum(duration_min) as mins_listened
, avg(popularity) as avg_popularity_listened
from Listening_History 
where played_at_date > '{two_weeks_ago}'
group by playlist_id
order by 4 desc
""", con)
print("Shape: ", df2.shape)
df2.head(10)


Shape:  (28, 5)


,playlist_id,num_songs_listened,unique_songs_listened,mins_listened,avg_popularity_listened
0,spotify:playlist:4teNNy1rJvRRa2qyL1Yj0A,154,70,665.416400,38.623377
1,None,163,117,655.015667,55.693252
2,spotify:artist:52qKfVcIV4GS8A8Vay2xtt,98,71,349.657917,53.959184
3,spotify:artist:70nxnxEqDQIEWneRjg2Q4O,42,42,166.747300,22.642857
4,spotify:playlist:6M56XPSUSRXBrm6P2CwFKF,35,34,150.543867,35.771429
5,spotify:playlist:37i9dQZF1EUMDoJuT8yJsl,35,23,145.084767,44.514286
6,spotify:album:7HgVH7ChzqayHdf1eAi9c1,36,16,103.418350,64.444444
7,spotify:playlist:37i9dQZF1DX5vMTfJy5XKE,29,23,100.372083,59.068966
8,spotify:playlist:334z5MApwRc1ERcUaGdeCB,23,23,80.346633,58.913043
9,spotify:playlist:1SrcegRRH9Cheuz5oM2BhE,20,18,68.794300,61.200000


## All Together 
Playlist info: 
 - how many minutes from playlist were listened to 
 - top playlists details ~ number of songs in playlist, length of playlist (songs, minutes), playlist first and last song added/date 
 
 

In [279]:
def parse_artist_details(spotify, artist_id):
    artist_call = spotify.artist(artist_id)
    out = {}
    out['Name'] = artist_call['name']
    out['Num Tracks'] = 'Followers: ' + str(artist_call['followers']['total'])
    out['Time (mins)'] = 'Popularity: ' + str(artist_call['popularity'])
    out['First Song'] = str(artist_call['genres'])[1:-1]
    out['Last Song'] = None
    out['Type'] = 'artist'
    return(pd.Series(out))

In [280]:
def parse_album_details(spotify, album_id):
    album_call = spotify.album(album_id)
    out = {}
    num_artists = len(album_call['artists'])
    artist_part = album_call['artists'][0]['name'] if num_artists == 1\
        else album_call['artists'][0] + f' - {num_artists} artists' 
    out['Name'] = album_call['name'] + ' - '+ artist_part
    out['Num Tracks'] = album_call['total_tracks']
    out['Time (mins)'] = sum([x['duration_ms'] for x in album_call['tracks']['items']])/60000
    out['First Song'] = album_call['tracks']['items'][0]['name']
    out['Last Song'] = album_call['tracks']['items'][-1]['name']
    out['Type'] = 'album'
    return(pd.Series(out))


In [281]:
def parse_playlist_details(spotify, playlist_id):
    a_pl = spotify.user_playlist(user='malchemist02', playlist_id=playlist_id)
    out = {}
    out['Name'] = a_pl['name']
    out['Num Tracks'] = a_pl['tracks']['total']
    out['Time (mins)'] = sum([x['track']['duration_ms'] for x in a_pl['tracks']['items']])/60000
    
    # Get first and last songs 
    songs_added = pd.DataFrame([[x['track']['name'] + ' ' + x['added_at'][:10], x.get('added_at') ] 
                                for x in a_pl['tracks']['items']], 
                          columns=['Track Name w Date', 'Date Time Added'])
    first_last_songs = songs_added[(songs_added['Date Time Added'] == songs_added['Date Time Added'].min())
           | (songs_added['Date Time Added'] == songs_added['Date Time Added'].max())]
    first_last_songs = first_last_songs.sort_values('Date Time Added')
    out['First Song'] = first_last_songs.iloc[0]['Track Name w Date']
    out['Last Song'] = first_last_songs.iloc[1]['Track Name w Date']
    out['Type'] = 'playlist'
    return(pd.Series(out))

In [282]:
def parse_source_uri(spotify, uri):
    split_uri = uri.split(':')
    if split_uri[1] == 'playlist':
        details = parse_playlist_details(spotify, split_uri[2])
    elif split_uri[1] == 'album':
        details = parse_album_details(spotify, split_uri[2])
    elif split_uri[1] == 'artist':
        details = parse_artist_details(spotify, split_uri[2])
    else:
        fields = ['Name', 'Num Tracks', 'Time (mins)', 'First Song',
       'Last Song', 'Type']
        details = pd.Series({x : None for x in fields})
    details['uri'] = uri
    return(details)

In [285]:
source_details = pd.Series(df2['playlist_id'].dropna()).apply(lambda x:parse_source_uri(spotify, x))
source_details

,Name,Num Tracks,Time (mins),First Song,Last Song,Type,uri
0,My Shazam Tracks,57,244.032,Hablame 2021-10-04,LA FAMA (with The Weeknd) 2021-12-29,playlist,spotify:playlist:4teNNy1rJvRRa2qyL1Yj0A
2,Ice Nine Kills,Followers: 497414,Popularity: 70,"'metalcore', 'screamo'",None,artist,spotify:artist:52qKfVcIV4GS8A8Vay2xtt
3,Adolescent's Orquesta,Followers: 474587,Popularity: 71,"'latin', 'salsa', 'salsa venezolana', 'tropical'",None,artist,spotify:artist:70nxnxEqDQIEWneRjg2Q4O
4,Shazam Apr-Sept 2021,153,432.607,Amor Perfecto 2021-11-04,Everything Changes 2021-11-04,playlist,spotify:playlist:6M56XPSUSRXBrm6P2CwFKF
5,Your Top Songs 2021,100,397.417,ANTES QUE SALGA EL SOL 1970-01-01,Vete Pa Ya 1970-01-01,playlist,spotify:playlist:37i9dQZF1EUMDoJuT8yJsl
6,Evergreen - Pentatonix,14,39.5988,"It's Been A Long, Long Time",We Wish You A Merry Christmas,album,spotify:album:7HgVH7ChzqayHdf1eAi9c1
7,New Music Holiday,130,318.197,Merry Christmas 2021-12-24,Jingle Bell Rock 2021-12-24,playlist,spotify:playlist:37i9dQZF1DX5vMTfJy5XKE
8,Jan 2021,32,111.266,4:38am 2021-01-14,Iglesia Rumbera 2021-05-27,playlist,spotify:playlist:334z5MApwRc1ERcUaGdeCB
9,2020 Christmas Songs,20,63.8781,My Only Wish (This Year) 2020-11-30,Happy New Year - Instrumental 2020-12-30,playlist,spotify:playlist:1SrcegRRH9Cheuz5oM2BhE
10,Christmas Hits,117,319.837,All I Want for Christmas Is You 2021-12-16,Oh Santa! (feat. Ariana Grande & Jennifer Huds...,playlist,spotify:playlist:37i9dQZF1DX0Yxoavh5qJV


In [286]:
all_playlist_info = pd.merge(df2, source_details, left_on='playlist_id', right_on='uri', how='outer' )
all_playlist_info

,playlist_id,num_songs_listened,unique_songs_listened,mins_listened,avg_popularity_listened,Name,Num Tracks,Time (mins),First Song,Last Song,Type,uri
0,spotify:playlist:4teNNy1rJvRRa2qyL1Yj0A,154,70,665.416400,38.623377,My Shazam Tracks,57,244.032,Hablame 2021-10-04,LA FAMA (with The Weeknd) 2021-12-29,playlist,spotify:playlist:4teNNy1rJvRRa2qyL1Yj0A
1,None,163,117,655.015667,55.693252,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,spotify:artist:52qKfVcIV4GS8A8Vay2xtt,98,71,349.657917,53.959184,Ice Nine Kills,Followers: 497414,Popularity: 70,"'metalcore', 'screamo'",None,artist,spotify:artist:52qKfVcIV4GS8A8Vay2xtt
3,spotify:artist:70nxnxEqDQIEWneRjg2Q4O,42,42,166.747300,22.642857,Adolescent's Orquesta,Followers: 474587,Popularity: 71,"'latin', 'salsa', 'salsa venezolana', 'tropical'",None,artist,spotify:artist:70nxnxEqDQIEWneRjg2Q4O
4,spotify:playlist:6M56XPSUSRXBrm6P2CwFKF,35,34,150.543867,35.771429,Shazam Apr-Sept 2021,153,432.607,Amor Perfecto 2021-11-04,Everything Changes 2021-11-04,playlist,spotify:playlist:6M56XPSUSRXBrm6P2CwFKF
5,spotify:playlist:37i9dQZF1EUMDoJuT8yJsl,35,23,145.084767,44.514286,Your Top Songs 2021,100,397.417,ANTES QUE SALGA EL SOL 1970-01-01,Vete Pa Ya 1970-01-01,playlist,spotify:playlist:37i9dQZF1EUMDoJuT8yJsl
6,spotify:album:7HgVH7ChzqayHdf1eAi9c1,36,16,103.418350,64.444444,Evergreen - Pentatonix,14,39.5988,"It's Been A Long, Long Time",We Wish You A Merry Christmas,album,spotify:album:7HgVH7ChzqayHdf1eAi9c1
7,spotify:playlist:37i9dQZF1DX5vMTfJy5XKE,29,23,100.372083,59.068966,New Music Holiday,130,318.197,Merry Christmas 2021-12-24,Jingle Bell Rock 2021-12-24,playlist,spotify:playlist:37i9dQZF1DX5vMTfJy5XKE
8,spotify:playlist:334z5MApwRc1ERcUaGdeCB,23,23,80.346633,58.913043,Jan 2021,32,111.266,4:38am 2021-01-14,Iglesia Rumbera 2021-05-27,playlist,spotify:playlist:334z5MApwRc1ERcUaGdeCB
9,spotify:playlist:1SrcegRRH9Cheuz5oM2BhE,20,18,68.794300,61.200000,2020 Christmas Songs,20,63.8781,My Only Wish (This Year) 2020-11-30,Happy New Year - Instrumental 2020-12-30,playlist,spotify:playlist:1SrcegRRH9Cheuz5oM2BhE


In [293]:
all_playlist_info.columns

Index(['playlist_id', 'num_songs_listened', 'unique_songs_listened',
       'mins_listened', 'avg_popularity_listened', 'Name', 'Num Tracks',
       'Time (mins)', 'First Song', 'Last Song', 'Type', 'uri'],
      dtype='object')

In [292]:
all_playlist_info[['Name', 'unique_songs_listened', 'mins_listened', 'num_songs_listened'\
                       , 'Num Tracks', 'Time (mins)', 'First Song', 'Last Song','Type']]

,Name,Type,unique_songs_listened,mins_listened,num_songs_listened,Num Tracks,Time (mins),First Song,Last Song,Type
0,My Shazam Tracks,playlist,70,665.416400,154,57,244.032,Hablame 2021-10-04,LA FAMA (with The Weeknd) 2021-12-29,playlist
1,NaN,NaN,117,655.015667,163,NaN,NaN,NaN,NaN,NaN
2,Ice Nine Kills,artist,71,349.657917,98,Followers: 497414,Popularity: 70,"'metalcore', 'screamo'",None,artist
3,Adolescent's Orquesta,artist,42,166.747300,42,Followers: 474587,Popularity: 71,"'latin', 'salsa', 'salsa venezolana', 'tropical'",None,artist
4,Shazam Apr-Sept 2021,playlist,34,150.543867,35,153,432.607,Amor Perfecto 2021-11-04,Everything Changes 2021-11-04,playlist
5,Your Top Songs 2021,playlist,23,145.084767,35,100,397.417,ANTES QUE SALGA EL SOL 1970-01-01,Vete Pa Ya 1970-01-01,playlist
6,Evergreen - Pentatonix,album,16,103.418350,36,14,39.5988,"It's Been A Long, Long Time",We Wish You A Merry Christmas,album
7,New Music Holiday,playlist,23,100.372083,29,130,318.197,Merry Christmas 2021-12-24,Jingle Bell Rock 2021-12-24,playlist
8,Jan 2021,playlist,23,80.346633,23,32,111.266,4:38am 2021-01-14,Iglesia Rumbera 2021-05-27,playlist
9,2020 Christmas Songs,playlist,18,68.794300,20,20,63.8781,My Only Wish (This Year) 2020-11-30,Happy New Year - Instrumental 2020-12-30,playlist


In [291]:
all_playlist_info2 = all_playlist_info[['Name', 'Num Tracks', 'Time (mins)', 'First Song', 'Last Song','Type']].dropna()
all_playlist_info2

,Name,Num Tracks,Time (mins),First Song,Last Song,Type
0,My Shazam Tracks,57,244.032,Hablame 2021-10-04,LA FAMA (with The Weeknd) 2021-12-29,playlist
4,Shazam Apr-Sept 2021,153,432.607,Amor Perfecto 2021-11-04,Everything Changes 2021-11-04,playlist
5,Your Top Songs 2021,100,397.417,ANTES QUE SALGA EL SOL 1970-01-01,Vete Pa Ya 1970-01-01,playlist
6,Evergreen - Pentatonix,14,39.5988,"It's Been A Long, Long Time",We Wish You A Merry Christmas,album
7,New Music Holiday,130,318.197,Merry Christmas 2021-12-24,Jingle Bell Rock 2021-12-24,playlist
8,Jan 2021,32,111.266,4:38am 2021-01-14,Iglesia Rumbera 2021-05-27,playlist
9,2020 Christmas Songs,20,63.8781,My Only Wish (This Year) 2020-11-30,Happy New Year - Instrumental 2020-12-30,playlist
10,Christmas Hits,117,319.837,All I Want for Christmas Is You 2021-12-16,Oh Santa! (feat. Ariana Grande & Jennifer Huds...,playlist
11,El Juicio - Willie Colón,8,38.968,Ah-Ah / O-No,Pan Y Agua,album
12,Tierra de Mis Amores - Grupo Niche,3,13.9493,El Coco,Culebra,album


In [233]:
playlist_details.index

Index(['Name', 'Num Tracks', 'Time (mins)', 'First Song on PL',
       'Last Song on PL', 'Type'],
      dtype='object')

In [222]:
playlist_details = parse_source_uri(spotify, unique_playlists[0])
playlist_details

Playlist Name                        My Shazam Tracks
Num Tracks                                         57
Time (mins)                                   244.032
First Song                         Hablame 2021-10-04
Last Song        LA FAMA (with The Weeknd) 2021-12-29
Type                                         playlist
dtype: object

In [223]:
playlist_details = parse_source_uri(spotify, unique_playlists[1])
playlist_details

Name                El JuicioWillie Colón
Num Tracks                              8
Time (mins)                        38.968
First Song on PL             Ah-Ah / O-No
Last Song on PL                Pan Y Agua
Type                                album
dtype: object

## Practice using Artist API 

In [237]:
artist_call = spotify.artist('70nxnxEqDQIEWneRjg2Q4O')
artist_call

{'external_urls': {'spotify': 'https://open.spotify.com/artist/70nxnxEqDQIEWneRjg2Q4O'},
 'followers': {'href': None, 'total': 474587},
 'genres': ['latin', 'salsa', 'salsa venezolana', 'tropical'],
 'href': 'https://api.spotify.com/v1/artists/70nxnxEqDQIEWneRjg2Q4O',
 'id': '70nxnxEqDQIEWneRjg2Q4O',
 'images': [{'height': 640,
   'url': 'https://i.scdn.co/image/ab6761610000e5eb56b42c5e89d436ab6ba2f17e',
   'width': 640},
  {'height': 320,
   'url': 'https://i.scdn.co/image/ab6761610000517456b42c5e89d436ab6ba2f17e',
   'width': 320},
  {'height': 160,
   'url': 'https://i.scdn.co/image/ab6761610000f17856b42c5e89d436ab6ba2f17e',
   'width': 160}],
 'name': "Adolescent's Orquesta",
 'popularity': 71,
 'type': 'artist',
 'uri': 'spotify:artist:70nxnxEqDQIEWneRjg2Q4O'}

In [248]:
out = {}
out['Name'] = artist_call['name']
out['Num Tracks'] = 'Followers: ' + str(artist_call['followers']['total'])
out['Time'] = 'Popularity: ' + str(artist_call['popularity'])
out['First Song'] = str(artist_call['genres'])[1:-1]
out['Last Song'] = ''
out['Type'] = 'artist'
out

{'Name': "Adolescent's Orquesta",
 'Num Tracks': 'Followers: 474587',
 'Time': 'Popularity: 71',
 'First Song': "'latin', 'salsa', 'salsa venezolana', 'tropical'",
 'Last Song': '',
 'Type': 'artist'}

## Practice using Album API 

In [196]:
album_call = spotify.album('3VGeuVPBeITXJwWt2YUPZW')
album_call

{'album_type': 'album',
 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/7x5Slu7yTE5icZjNsc3OzW'},
   'href': 'https://api.spotify.com/v1/artists/7x5Slu7yTE5icZjNsc3OzW',
   'id': '7x5Slu7yTE5icZjNsc3OzW',
   'name': 'Willie Colón',
   'type': 'artist',
   'uri': 'spotify:artist:7x5Slu7yTE5icZjNsc3OzW'}],
 'available_markets': ['AD',
  'AE',
  'AG',
  'AL',
  'AM',
  'AO',
  'AR',
  'AT',
  'AU',
  'AZ',
  'BA',
  'BB',
  'BD',
  'BE',
  'BF',
  'BG',
  'BH',
  'BI',
  'BJ',
  'BN',
  'BO',
  'BR',
  'BS',
  'BT',
  'BW',
  'BY',
  'BZ',
  'CA',
  'CD',
  'CG',
  'CH',
  'CI',
  'CL',
  'CM',
  'CO',
  'CR',
  'CV',
  'CW',
  'CY',
  'CZ',
  'DE',
  'DJ',
  'DK',
  'DM',
  'DO',
  'DZ',
  'EC',
  'EE',
  'EG',
  'ES',
  'FI',
  'FJ',
  'FM',
  'FR',
  'GA',
  'GB',
  'GD',
  'GE',
  'GH',
  'GM',
  'GN',
  'GQ',
  'GR',
  'GT',
  'GW',
  'GY',
  'HK',
  'HN',
  'HR',
  'HT',
  'HU',
  'ID',
  'IE',
  'IL',
  'IN',
  'IQ',
  'IS',
  'IT',
  'JM',
  'JO',
  'JP

In [201]:
album_call.keys()

dict_keys(['album_type', 'artists', 'available_markets', 'copyrights', 'external_ids', 'external_urls', 'genres', 'href', 'id', 'images', 'label', 'name', 'popularity', 'release_date', 'release_date_precision', 'total_tracks', 'tracks', 'type', 'uri'])

In [202]:
album_call['artists']

[{'external_urls': {'spotify': 'https://open.spotify.com/artist/7x5Slu7yTE5icZjNsc3OzW'},
  'href': 'https://api.spotify.com/v1/artists/7x5Slu7yTE5icZjNsc3OzW',
  'id': '7x5Slu7yTE5icZjNsc3OzW',
  'name': 'Willie Colón',
  'type': 'artist',
  'uri': 'spotify:artist:7x5Slu7yTE5icZjNsc3OzW'}]

In [206]:
album_call['tracks']['items']

[{'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/7x5Slu7yTE5icZjNsc3OzW'},
    'href': 'https://api.spotify.com/v1/artists/7x5Slu7yTE5icZjNsc3OzW',
    'id': '7x5Slu7yTE5icZjNsc3OzW',
    'name': 'Willie Colón',
    'type': 'artist',
    'uri': 'spotify:artist:7x5Slu7yTE5icZjNsc3OzW'},
   {'external_urls': {'spotify': 'https://open.spotify.com/artist/7opp16lU7VM3l2WBdGMYHP'},
    'href': 'https://api.spotify.com/v1/artists/7opp16lU7VM3l2WBdGMYHP',
    'id': '7opp16lU7VM3l2WBdGMYHP',
    'name': 'Héctor Lavoe',
    'type': 'artist',
    'uri': 'spotify:artist:7opp16lU7VM3l2WBdGMYHP'}],
  'available_markets': ['AD',
   'AE',
   'AG',
   'AL',
   'AM',
   'AO',
   'AR',
   'AT',
   'AU',
   'AZ',
   'BA',
   'BB',
   'BD',
   'BE',
   'BF',
   'BG',
   'BH',
   'BI',
   'BJ',
   'BN',
   'BO',
   'BR',
   'BS',
   'BT',
   'BW',
   'BY',
   'BZ',
   'CA',
   'CD',
   'CG',
   'CH',
   'CI',
   'CL',
   'CM',
   'CO',
   'CR',
   'CV',
   'CW',
   'CY',
   'CZ',

In [210]:
album_call['tracks']['items'][0]['name']

'Ah-Ah / O-No'

In [211]:
album_call['tracks']['items'][-1]['name']

'Pan Y Agua'

## Practice using Playlist API

In [177]:
songs_added = pd.DataFrame([[x['track']['name'] + x['added_at'][:10], x.get('added_at') ] 
                                for x in a_pl['tracks']['items']], 
                          columns=['Track Name w Date', 'Date Time Added'])
songs_added.sort_values('Date Time Added')



,Track Name w Date,Date Time Added
4,Hablame2021-10-04,2021-10-04T22:57:51Z
5,Guararé2021-10-11,2021-10-11T13:40:41Z
0,Solamente2021-10-19,2021-10-19T17:00:16Z
1,seaside_demo2021-10-19,2021-10-19T17:16:26Z
6,Pa'lla Voy2021-10-24,2021-10-24T16:00:56Z
7,Bachatica2021-10-24,2021-10-24T17:16:46Z
3,Nothing Else2021-10-25,2021-10-25T21:32:31Z
2,Mr. Perfectly Fine (Taylor’s Version) (From Th...,2021-10-25T21:44:37Z
8,I'll Always Love You2021-10-28,2021-10-28T17:35:45Z
9,SEJODIOTO2021-10-29,2021-10-29T04:22:08Z


In [180]:
first_last_songs = songs_added[(songs_added['Date Time Added'] == songs_added['Date Time Added'].min())
           | (songs_added['Date Time Added'] == songs_added['Date Time Added'].max())]
first_last_songs.iloc[0]['Track Name w Date']

'Hablame2021-10-04'

In [161]:
# a_pl = spotify.user_playlist(user='malchemist02', playlist_id='4teNNy1rJvRRa2qyL1Yj0A')
a_pl['tracks']['items'][0]['track']['name']

'Solamente'

In [131]:
a_pl.keys()

dict_keys(['collaborative', 'description', 'external_urls', 'followers', 'href', 'id', 'images', 'name', 'owner', 'primary_color', 'public', 'snapshot_id', 'tracks', 'type', 'uri'])

In [136]:
a_pl['tracks']['total']

57

In [120]:
a_pl = spotify.user_playlist_tracks(user='malchemist02', playlist_id='4teNNy1rJvRRa2qyL1Yj0A')
a_pl

{'href': 'https://api.spotify.com/v1/playlists/4teNNy1rJvRRa2qyL1Yj0A/tracks?offset=0&limit=100&additional_types=track',
 'items': [{'added_at': '2021-10-19T17:00:16Z',
   'added_by': {'external_urls': {'spotify': 'https://open.spotify.com/user/malchemist02'},
    'href': 'https://api.spotify.com/v1/users/malchemist02',
    'id': 'malchemist02',
    'type': 'user',
    'uri': 'spotify:user:malchemist02'},
   'is_local': False,
   'primary_color': None,
   'track': {'album': {'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/2kCO8LXN1usaOPL3iEE28I'},
       'href': 'https://api.spotify.com/v1/artists/2kCO8LXN1usaOPL3iEE28I',
       'id': '2kCO8LXN1usaOPL3iEE28I',
       'name': 'Tai Verdes',
       'type': 'artist',
       'uri': 'spotify:artist:2kCO8LXN1usaOPL3iEE28I'}],
     'available_markets': ['US'],
     'external_urls': {'spotify': 'https://open.spotify.com/album/6E8lxwX7KMAO9nCx4A5tAR'},
     'href': 'https://api.spotify.com/v

In [122]:
a_pl.keys()

dict_keys(['href', 'items', 'limit', 'next', 'offset', 'previous', 'total'])

In [128]:
a_pl['total']

57

In [123]:
a_pl['items'][0].keys()

dict_keys(['added_at', 'added_by', 'is_local', 'primary_color', 'track', 'video_thumbnail'])

In [124]:
a_pl['items'][0]['track']

{'album': {'album_type': 'album',
  'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/2kCO8LXN1usaOPL3iEE28I'},
    'href': 'https://api.spotify.com/v1/artists/2kCO8LXN1usaOPL3iEE28I',
    'id': '2kCO8LXN1usaOPL3iEE28I',
    'name': 'Tai Verdes',
    'type': 'artist',
    'uri': 'spotify:artist:2kCO8LXN1usaOPL3iEE28I'}],
  'available_markets': ['US'],
  'external_urls': {'spotify': 'https://open.spotify.com/album/6E8lxwX7KMAO9nCx4A5tAR'},
  'href': 'https://api.spotify.com/v1/albums/6E8lxwX7KMAO9nCx4A5tAR',
  'id': '6E8lxwX7KMAO9nCx4A5tAR',
  'images': [{'height': 640,
    'url': 'https://i.scdn.co/image/ab67616d0000b2737bee23b5177b81131d7d98e9',
    'width': 640},
   {'height': 300,
    'url': 'https://i.scdn.co/image/ab67616d00001e027bee23b5177b81131d7d98e9',
    'width': 300},
   {'height': 64,
    'url': 'https://i.scdn.co/image/ab67616d000048517bee23b5177b81131d7d98e9',
    'width': 64}],
  'name': 'TV',
  'release_date': '2021-05-20',
  'release_date_prec

In [112]:
all_playlist_df = pd.DataFrame(all_pls_sum, columns=['Playlist Name', 'Number of Songs', 'ID', 'uri'])

In [92]:
df1 = pd.read_sql(f""" 
select * from Listening_History lh 
left join Playlist_IDs pid
on lh.playlist_id = pid.uri
where played_at_date > '{two_weeks_ago}'

""", con)
df1['Playlist Name'].fillna('NA', inplace=True)
df1

,index,name,artist_name,played_at_date,played_at_time,duration_min,popularity,song_uri,artist_id,playlist_id,after_ts,Playlist Name,Number of Songs,ID,uri
0,0,Bella Ciao - Música Original de la Serie la Ca...,Manu Pilas,2021-12-05,09:43,2.330733,71,spotify:track:3lWzVNe1yFZlkeBBzUuZYu,spotify:artist:2TJHmhbmT7L3gw2NKyDTHh,None,1638711834460,NA,NaN,None,None
1,1,Bella Ciao - Música Original de la Serie la Ca...,Manu Pilas,2021-12-05,09:43,2.330733,71,spotify:track:3lWzVNe1yFZlkeBBzUuZYu,spotify:artist:2TJHmhbmT7L3gw2NKyDTHh,None,1638711834460,NA,NaN,None,None
2,0,Bella Ciao - Música Original de la Serie la Ca...,Manu Pilas,2021-12-05,13:21,2.330733,71,spotify:track:3lWzVNe1yFZlkeBBzUuZYu,spotify:artist:2TJHmhbmT7L3gw2NKyDTHh,None,1638724911098,NA,NaN,None,None
3,0,Santa Baby,Eartha Kitt,2021-12-05,18:49,3.441767,75,spotify:track:1vZKP9XURuqMp1SpXGnoyb,spotify:artist:1AwO9pWEBSBoWdEZu28XDC,None,1638744583446,NA,NaN,None,None
4,0,Barrunto,Willie Colón,2021-12-05,20:14,5.602000,41,spotify:track:0oCentzpU5hprEWJtggVIc,spotify:artist:7x5Slu7yTE5icZjNsc3OzW,spotify:playlist:4teNNy1rJvRRa2qyL1Yj0A,1638749658272,My Shazam Tracks,52.0,4teNNy1rJvRRa2qyL1Yj0A,spotify:playlist:4teNNy1rJvRRa2qyL1Yj0A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
665,1,Algo Que Se Quede,Grupo Niche,2021-12-24,16:05,3.668917,68,spotify:track:1JXcB149QtbcIpDUA6YZOB,spotify:artist:1zng9JZpblpk48IPceRWs8,None,1640376523955,NA,NaN,None,None
666,2,Virgen,Adolescent's Orquesta,2021-12-24,16:01,4.529433,79,spotify:track:6srBp90EwADKAF7PorGiUC,spotify:artist:70nxnxEqDQIEWneRjg2Q4O,None,1640376523955,NA,NaN,None,None
667,3,Aquel Lugar,Adolescent's Orquesta,2021-12-24,15:57,3.856833,65,spotify:track:16c6awlmGfeusU7zoVu5yc,spotify:artist:70nxnxEqDQIEWneRjg2Q4O,None,1640376523955,NA,NaN,None,None
668,0,Step Into Christmas,Elton John,2021-12-25,00:11,4.539883,88,spotify:track:6sBWmE23q6xQHlnEZ8jYPT,spotify:artist:3PhoLpVuITZKcymswpck5b,None,1640405493384,NA,NaN,None,None


In [93]:
df1['Playlist Name'].value_counts()

NA                                             409
My Shazam Tracks                               149
Shazam Apr-Sept 2021                            35
Jan 2021                                        23
2020 Christmas Songs                            20
Meghan Trainor – A Very Trainor Christmas 2     10
December 2020                                    9
Feb 2021                                         8
Decent Salsa 2                                   6
Bailá por la mañana                              1
Name: Playlist Name, dtype: int64

In [108]:
def playlist_func(df):
    out = {}
    out['Count'] = df.shape[0]
    out['Songs in Playlist'] = df['Number of Songs'].max()
    out['Total duration'] = df['duration_min'].sum()
    out['popularity avg'] = df['popularity'].mean().round(2)
    
    out_series = pd.Series(out)
    return(out_series)

In [109]:
df1.groupby('Playlist Name').apply(lambda x: playlist_func(x)).sort_values('Count', ascending=False)

,Count,Songs in Playlist,Total duration,popularity avg
Playlist Name,,,,
NA,409.0,NaN,1530.203100,55.71
My Shazam Tracks,149.0,52.0,644.111267,38.60
Shazam Apr-Sept 2021,35.0,153.0,150.543867,35.77
Jan 2021,23.0,32.0,80.346633,58.91
2020 Christmas Songs,20.0,20.0,68.794300,61.20
Meghan Trainor – A Very Trainor Christmas 2,10.0,18.0,31.124617,61.90
December 2020,9.0,23.0,32.027717,62.67
Feb 2021,8.0,15.0,27.389117,62.62
Decent Salsa 2,6.0,27.0,29.141533,38.83


In [94]:
con.commit()
con.close()

## Practice Using API 

In [20]:
playlists1 = spotify.user_playlists('malchemist02', limit=50, offset=0)
playlists1

{'href': 'https://api.spotify.com/v1/users/malchemist02/playlists?offset=0&limit=50',
 'items': [{'collaborative': False,
   'description': '',
   'external_urls': {'spotify': 'https://open.spotify.com/playlist/488BqUhlYu7wCPJVVO2bp6'},
   'href': 'https://api.spotify.com/v1/playlists/488BqUhlYu7wCPJVVO2bp6',
   'id': '488BqUhlYu7wCPJVVO2bp6',
   'images': [{'height': 640,
     'url': 'https://i.scdn.co/image/ab67616d0000b273cd5c29f3186667fbf3646f71',
     'width': 640}],
   'name': 'Dec 2021',
   'owner': {'display_name': 'Malcolm Taylor',
    'external_urls': {'spotify': 'https://open.spotify.com/user/malchemist02'},
    'href': 'https://api.spotify.com/v1/users/malchemist02',
    'id': 'malchemist02',
    'type': 'user',
    'uri': 'spotify:user:malchemist02'},
   'primary_color': None,
   'public': True,
   'snapshot_id': 'Miw4ZTRlZDVkYmM2YWNlZGFhMzYwMjg0NDg4MTg2MDc3NDk3Y2FjYjY4',
   'tracks': {'href': 'https://api.spotify.com/v1/playlists/488BqUhlYu7wCPJVVO2bp6/tracks',
    'total

In [21]:
playlists1.keys()

dict_keys(['href', 'items', 'limit', 'next', 'offset', 'previous', 'total'])

In [23]:
total = playlists1['total']
total

349

In [29]:
raw_playlist_objs = []
for x in range(int(total/50)+1):
    offset1 = x*50
    print(offset1)
    raw_playlist_objs.append(spotify.user_playlists('malchemist02', limit=50, offset=offset1))

0
50
100
150
200
250
300


In [11]:
playlists['items'][0].keys()

dict_keys(['collaborative', 'description', 'external_urls', 'href', 'id', 'images', 'name', 'owner', 'primary_color', 'public', 'snapshot_id', 'tracks', 'type', 'uri'])

In [34]:
playlists['items'][1]

{'collaborative': False,
 'description': '',
 'external_urls': {'spotify': 'https://open.spotify.com/playlist/2OMofo56eFKCUDtQLOrrzf'},
 'href': 'https://api.spotify.com/v1/playlists/2OMofo56eFKCUDtQLOrrzf',
 'id': '2OMofo56eFKCUDtQLOrrzf',
 'images': [{'height': 640,
   'url': 'https://i.scdn.co/image/ab67616d0000b2736a83a7f1411a86b69c5fb0b9',
   'width': 640}],
 'name': 'Bailá por la mañana',
 'owner': {'display_name': 'Malcolm Taylor',
  'external_urls': {'spotify': 'https://open.spotify.com/user/malchemist02'},
  'href': 'https://api.spotify.com/v1/users/malchemist02',
  'id': 'malchemist02',
  'type': 'user',
  'uri': 'spotify:user:malchemist02'},
 'primary_color': None,
 'public': True,
 'snapshot_id': 'MixjYzgyYjMyM2U4MTg1OWNjZGJlYzdiZDMxNjMwOTgzMjc2ZGY0NDgx',
 'tracks': {'href': 'https://api.spotify.com/v1/playlists/2OMofo56eFKCUDtQLOrrzf/tracks',
  'total': 24},
 'type': 'playlist',
 'uri': 'spotify:playlist:2OMofo56eFKCUDtQLOrrzf'}

In [ ]:
pd.DataFrame(pla)

In [73]:
all_pls_sum = []
for playlist in raw_playlist_objs:
    all_pls_sum.extend([(x['name'], x['tracks']['total'], x['id'], x['uri']) for x in playlist['items']])

In [74]:
pd.DataFrame(all_pls_sum, columns=['Playlist Name', 'Number of Songs', 'ID', 'uri'])

,Playlist Name,Number of Songs,ID,uri
0,Dec 2021,1,488BqUhlYu7wCPJVVO2bp6,spotify:playlist:488BqUhlYu7wCPJVVO2bp6
1,Bailá por la mañana,24,2OMofo56eFKCUDtQLOrrzf,spotify:playlist:2OMofo56eFKCUDtQLOrrzf
2,My Shazam Tracks,52,4teNNy1rJvRRa2qyL1Yj0A,spotify:playlist:4teNNy1rJvRRa2qyL1Yj0A
3,Shazam Apr-Sept 2021,153,6M56XPSUSRXBrm6P2CwFKF,spotify:playlist:6M56XPSUSRXBrm6P2CwFKF
4,Fall 2021,11,00BLkNPEe8tPdJ61wIyNbQ,spotify:playlist:00BLkNPEe8tPdJ61wIyNbQ
...,...,...,...,...
344,Tom Hanks - The Polar Express - Original Motio...,16,5au4JY94u0QgfDCvxTlPwM,spotify:playlist:5au4JY94u0QgfDCvxTlPwM
345,Billy Joel - The Hits,19,5hcRHJFsNsRGuWeCSs2vQ3,spotify:playlist:5hcRHJFsNsRGuWeCSs2vQ3
346,Country Strong - Country Strong,30,0hSDJvLCo80WENg69qoorX,spotify:playlist:0hSDJvLCo80WENg69qoorX
347,Elzhi - Elmatic,12,03RpRT4jq5VeyM630P13aE,spotify:playlist:03RpRT4jq5VeyM630P13aE


In [75]:
a_pl = spotify.user_playlist_tracks(user='malchemist02', playlist_id='4teNNy1rJvRRa2qyL1Yj0A')
a_pl

{'href': 'https://api.spotify.com/v1/playlists/4teNNy1rJvRRa2qyL1Yj0A/tracks?offset=0&limit=100&additional_types=track',
 'items': [{'added_at': '2021-10-19T17:00:16Z',
   'added_by': {'external_urls': {'spotify': 'https://open.spotify.com/user/malchemist02'},
    'href': 'https://api.spotify.com/v1/users/malchemist02',
    'id': 'malchemist02',
    'type': 'user',
    'uri': 'spotify:user:malchemist02'},
   'is_local': False,
   'primary_color': None,
   'track': {'album': {'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/2kCO8LXN1usaOPL3iEE28I'},
       'href': 'https://api.spotify.com/v1/artists/2kCO8LXN1usaOPL3iEE28I',
       'id': '2kCO8LXN1usaOPL3iEE28I',
       'name': 'Tai Verdes',
       'type': 'artist',
       'uri': 'spotify:artist:2kCO8LXN1usaOPL3iEE28I'}],
     'available_markets': ['US'],
     'external_urls': {'spotify': 'https://open.spotify.com/album/6E8lxwX7KMAO9nCx4A5tAR'},
     'href': 'https://api.spotify.com/v

In [76]:
a_pl['items']

[{'added_at': '2021-10-19T17:00:16Z',
  'added_by': {'external_urls': {'spotify': 'https://open.spotify.com/user/malchemist02'},
   'href': 'https://api.spotify.com/v1/users/malchemist02',
   'id': 'malchemist02',
   'type': 'user',
   'uri': 'spotify:user:malchemist02'},
  'is_local': False,
  'primary_color': None,
  'track': {'album': {'album_type': 'album',
    'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/2kCO8LXN1usaOPL3iEE28I'},
      'href': 'https://api.spotify.com/v1/artists/2kCO8LXN1usaOPL3iEE28I',
      'id': '2kCO8LXN1usaOPL3iEE28I',
      'name': 'Tai Verdes',
      'type': 'artist',
      'uri': 'spotify:artist:2kCO8LXN1usaOPL3iEE28I'}],
    'available_markets': ['US'],
    'external_urls': {'spotify': 'https://open.spotify.com/album/6E8lxwX7KMAO9nCx4A5tAR'},
    'href': 'https://api.spotify.com/v1/albums/6E8lxwX7KMAO9nCx4A5tAR',
    'id': '6E8lxwX7KMAO9nCx4A5tAR',
    'images': [{'height': 640,
      'url': 'https://i.scdn.co/image/ab67616d0

In [275]:
con.commit()
con.close()